In [1]:
import trimesh
from plyfile import PlyData, PlyElement, PlyProperty
import numpy as np
import laspy as lp

In [2]:
point_cloud=lp.read("davis_big.las")

In [3]:
point_cloud

<LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 169577267 points, 2 vlrs)>

In [4]:
list(point_cloud.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'red',
 'green',
 'blue']

In [5]:
import pandas as pd
point_data = np.stack([point_cloud.X, point_cloud.Y, point_cloud.Z, point_cloud.intensity,point_cloud.classification], axis=0).transpose((1, 0))
point_data

array([[ 61011822, 427032655,      1041,     50068,         2],
       [ 61011441, 427032652,      1041,     48626,         2],
       [ 61010827, 427032396,      1041,     47163,         2],
       ...,
       [ 60398563, 426691795,      2046,     50571,         2],
       [ 60398244, 426691759,      2046,     48299,         2],
       [ 60398311, 426691797,      2046,     49304,         2]],
      dtype=int32)

In [6]:
df = pd.DataFrame(data=point_data, columns=['X','Y','Z','reflectance','Class'])
df

,X,Y,Z,reflectance,Class
0,61011822,427032655,1041,50068,2
1,61011441,427032652,1041,48626,2
2,61010827,427032396,1041,47163,2
3,61011271,427032644,1047,49806,2
4,61011634,427032847,1040,50898,2
...,...,...,...,...,...
169577262,60398425,426691717,2045,48910,2
169577263,60398495,426691757,2047,50177,2
169577264,60398563,426691795,2046,50571,2
169577265,60398244,426691759,2046,48299,2


In [7]:
df.describe()

,X,Y,Z,reflectance,Class
count,1.695773e+08,1.695773e+08,1.695773e+08,1.695773e+08,1.695773e+08
mean,6.068609e+07,4.267067e+08,1.840397e+03,4.945400e+04,1.748637e+00
std,1.926541e+05,1.916313e+05,8.269772e+02,7.536899e+03,7.712580e-01
min,6.033885e+07,4.263662e+08,-1.365600e+04,0.000000e+00,1.000000e+00
25%,6.051938e+07,4.265418e+08,1.538000e+03,4.622400e+04,1.000000e+00
50%,6.069257e+07,4.267121e+08,1.813000e+03,5.216500e+04,2.000000e+00
75%,6.085290e+07,4.268679e+08,2.037000e+03,5.450300e+04,2.000000e+00
max,6.102151e+07,4.270475e+08,2.080060e+05,6.553500e+04,2.000000e+01


In [8]:
# df = df[df.Z < df.Z.quantile(.999)]
# df = df[df.Z > df.Z.quantile(.001)]
# df

In [9]:
df.describe()

,X,Y,Z,reflectance,Class
count,1.695773e+08,1.695773e+08,1.695773e+08,1.695773e+08,1.695773e+08
mean,6.068609e+07,4.267067e+08,1.840397e+03,4.945400e+04,1.748637e+00
std,1.926541e+05,1.916313e+05,8.269772e+02,7.536899e+03,7.712580e-01
min,6.033885e+07,4.263662e+08,-1.365600e+04,0.000000e+00,1.000000e+00
25%,6.051938e+07,4.265418e+08,1.538000e+03,4.622400e+04,1.000000e+00
50%,6.069257e+07,4.267121e+08,1.813000e+03,5.216500e+04,2.000000e+00
75%,6.085290e+07,4.268679e+08,2.037000e+03,5.450300e+04,2.000000e+00
max,6.102151e+07,4.270475e+08,2.080060e+05,6.553500e+04,2.000000e+01


In [10]:
df['reflectance'] = 0.0
df

,X,Y,Z,reflectance,Class
0,61011822,427032655,1041,0.0,2
1,61011441,427032652,1041,0.0,2
2,61010827,427032396,1041,0.0,2
3,61011271,427032644,1047,0.0,2
4,61011634,427032847,1040,0.0,2
...,...,...,...,...,...
169577262,60398425,426691717,2045,0.0,2
169577263,60398495,426691757,2047,0.0,2
169577264,60398563,426691795,2046,0.0,2
169577265,60398244,426691759,2046,0.0,2


In [11]:
df.Class.value_counts()

2     120856481
1      48146421
9        340727
17       137442
7         48951
20        36774
18        10471
Name: Class, dtype: int64

<h2> Scale convert to Dale's scale
    

In [12]:
### Assign Control variables based on Dales training data set 5085_54320.ply
control_x_min = 250.024994
control_x_max = 750.075012

control_y_min = 250.020004
control_y_max = 750.059998

control_z_min = 24.190001
control_z_max = 72.570000

In [13]:
##### Assign LAS min - max difference from control 5085_54320.las]

control_diff_x = 50005
control_diff_y = 50004
control_diff_z = 4838

##### For new data feed in this will compare the ratio of our control diff then apply the adjusted ratio to the control min_max variables

In [14]:
### new data las diff

x_diff = df.X.max() - df.X.min()
y_diff = df.Y.max() - df.Y.min()
z_diff = df.Z.max() - df.Z.min()


### adjusted ratio

x_ratio = x_diff / control_diff_x
y_ratio = y_diff / control_diff_y
z_ratio = z_diff / control_diff_z


In [15]:
##### Calculate new data min-max 

x_min = control_x_min * x_ratio
x_max = control_x_max * x_ratio

y_min = control_y_min * y_ratio
y_max = control_y_max * y_ratio

z_min = control_z_min * z_ratio
z_max = control_z_max * z_ratio


In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler(feature_range=(x_min, x_max))
scaler_y = MinMaxScaler(feature_range=(y_min, y_max))
scaler_z = MinMaxScaler(feature_range=(z_min, z_max))

In [17]:
scaler_x.fit_transform(df[['X', 'Class']])

array([[10143.05016034,  3772.59466786],
       [10139.2401602 ,  3772.59466786],
       [10133.10015998,  3772.59466786],
       ...,
       [ 4010.45993958,  3772.59466786],
       [ 4007.26993947,  3772.59466786],
       [ 4007.93993949,  3772.59466786]])

In [18]:
scaler_y.fit_transform(df[['Y', 'Class']])

array([[10071.16497453,  3765.00715546],
       [10071.13497453,  3765.00715546],
       [10068.57497456,  3765.00715546],
       ...,
       [ 6662.56501543,  3765.00715546],
       [ 6662.20501543,  3765.00715546],
       [ 6662.58501543,  3765.00715546]])

In [19]:
scaler_z.fit_transform(df[['Z', 'Class']])

array([[1255.28004278, 1224.97425393],
       [1255.28004278, 1224.97425393],
       [1255.28004278, 1224.97425393],
       ...,
       [1265.33004257, 1224.97425393],
       [1265.33004257, 1224.97425393],
       [1265.33004257, 1224.97425393]])

In [20]:
df['X'] = scaler_x.fit_transform(df[['X', 'Class']])[:,0]
df['Y'] = scaler_y.fit_transform(df[['Y', 'Class']])[:,0]
df['Z'] = scaler_z.fit_transform(df[['Z', 'Class']])[:,0]
df

,X,Y,Z,reflectance,Class
0,10143.050160,10071.164975,1255.280043,0.0,2
1,10139.240160,10071.134975,1255.280043,0.0,2
2,10133.100160,10068.574975,1255.280043,0.0,2
3,10137.540160,10071.054975,1255.340043,0.0,2
4,10141.170160,10073.084975,1255.270043,0.0,2
...,...,...,...,...,...
169577262,4009.079940,6661.785015,1265.320043,0.0,2
169577263,4009.779940,6662.185015,1265.340043,0.0,2
169577264,4010.459940,6662.565015,1265.330043,0.0,2
169577265,4007.269939,6662.205015,1265.330043,0.0,2


<h2> Convert to Ply

In [21]:
arr = df.to_numpy()

In [22]:
some_arr = np.array([tuple(i) for i in arr], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'),('reflectance', 'f4'),('class', 'f4')])

KeyboardInterrupt: 

In [ ]:
some_arr

In [ ]:
el = PlyElement.describe(some_arr, 'vertex')

In [ ]:
%%time
PlyData([el], text=True).write('davis_big.ply')